<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/preprocessing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download datasets
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/event2semantic_vec.json'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_normal'
! wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_abnormal'

--2023-07-11 15:43:52--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/event2semantic_vec.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259484 (253K) [text/plain]
Saving to: ‘event2semantic_vec.json’

event2semantic_vec. 100%[===================>] 253.40K  --.-KB/s    in 0.03s   

2023-07-11 15:43:52 (7.85 MB/s) - ‘event2semantic_vec.json’ saved [259484/259484]

--2023-07-11 15:43:52--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [2]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from tqdm import tqdm


In [4]:
with open('/content/event2semantic_vec.json', 'r') as load_f:
  event2semantic_vec  = json.load(load_f) #it is a dictionary, for exaple, first template is '0'and each templates is 300 demension

In [6]:
len(event2semantic_vec)

29

In [11]:
len(event2semantic_vec['0'])

300

In [16]:
Sequential_pattern = [4, 4, 4, 21, 10, 8, 10, 8, 10, 8]
Quantitative_pattern = [0] * 28
log_counter = Counter(Sequential_pattern)
print(log_counter)

for key in log_counter:
    Quantitative_pattern[key] = log_counter[key]

print(Quantitative_pattern)

Counter({4: 3, 10: 3, 8: 3, 21: 1})
[0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [18]:
Sequential_pattern = [4, 4, 4, 21, 10, 8, 10, 8, 10, 8]
Sequential_pattern = np.array(Sequential_pattern)[:, np.newaxis]
Sequential_pattern.shape

(10, 1)

In [20]:
data_dir = '/content/hdfs_train'
window_size = 10
num_sessions = 0
result_logs = {}
result_logs['Sequentials'] = []
result_logs['Quantitatives'] = []
result_logs['Semantics'] = []
labels = []

with open('/content/hdfs_train', 'r') as f:

    for line in f:
        num_sessions += 1
        line = [int(i) - 1 for i in line.strip().split()] # we substract by one from templates index for starting from zero

        for i in range(len(line) - window_size):
            Sequential_pattern = list(line[i:i + window_size])

            Quantitative_pattern = [0] * 28
            log_counter = Counter(Sequential_pattern)
            for key in log_counter:
                Quantitative_pattern[key] = log_counter[key]

            Semantic_pattern = []
            for event in Sequential_pattern:
                if event == 0:
                    Semantic_pattern.append([-1] * 300)
                else:
                    Semantic_pattern.append(event2semantic_vec[str(event - 1)])

            Sequential_pattern = np.array(Sequential_pattern)[:, np.newaxis]
            Quantitative_pattern = np.array(Quantitative_pattern)[:, np.newaxis]
            result_logs['Sequentials'].append(Sequential_pattern)
            result_logs['Quantitatives'].append(Quantitative_pattern)
            result_logs['Semantics'].append(Semantic_pattern)
            labels.append(line[i + window_size])

print('File {}, number of sessions {}'.format(data_dir, num_sessions))
print('File {}, number of seqs {}'.format(data_dir,len(result_logs['Sequentials'])))

File /content/hdfs_train, number of sessions 4855
File /content/hdfs_train, number of seqs 46575


In [27]:
# first line of hdfs_train

with open('/content/hdfs_train', 'r') as f:

    for line in f:
        line = [int(i) - 1 for i in line.strip().split()]
        break
print(line)

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8, 25, 25, 25, 22, 22, 22, 20, 20, 20]


In [25]:
labels[0]

25

In [22]:
result_logs['Sequentials'][0]

array([[ 4],
       [ 4],
       [ 4],
       [21],
       [10],
       [ 8],
       [10],
       [ 8],
       [10],
       [ 8]])

In [23]:
result_logs['Quantitatives'][0]

array([[0],
       [0],
       [0],
       [0],
       [3],
       [0],
       [0],
       [0],
       [3],
       [0],
       [3],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [24]:
result_logs['Semantics'][0]

[[0.005266666666666668,
  0.021233333333333333,
  0.007733333333333328,
  0.09166666666666666,
  -0.024633333333333333,
  0.03889999999999999,
  0.0028666666666666667,
  -0.037366666666666666,
  -0.016533333333333334,
  0.026366666666666667,
  0.0027666666666666664,
  0.0002000000000000001,
  -0.009666666666666664,
  0.05263333333333333,
  0.008433333333333333,
  0.05453333333333333,
  -0.012199999999999999,
  -0.0011333333333333317,
  -0.049699999999999994,
  -0.020999999999999998,
  0.028266666666666662,
  0.0012666666666666677,
  -0.03263333333333333,
  -0.023633333333333333,
  -0.029333333333333333,
  -0.04243333333333334,
  0.028166666666666666,
  0.025466666666666672,
  -0.010333333333333335,
  0.026933333333333333,
  -0.0009666666666666658,
  -0.019366666666666664,
  -0.033466666666666665,
  -0.019399999999999994,
  -0.031266666666666665,
  0.029633333333333324,
  0.06706666666666666,
  -0.0022333333333333302,
  -0.007933333333333332,
  0.012333333333333333,
  0.0081666666666666